# Script showing impact of refraction in the Agulhas current
Using surface currents from ESAs GlobCurrent project: http://globcurrent.ifremer.fr/

In [1]:
import numpy as np
import xarray as xa
import matplotlib.pyplot as plt
import cmocean
import datetime
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pandas as pd

import sys,os
testdir = os.path.dirname(os.getcwd() + '/')
srcdir = '..'
sys.path.insert(0, os.path.abspath(os.path.join(testdir, srcdir)))

from ocean_wave_tracing import Wave_tracing


In [2]:
#Read data
URI = 'http://tds0.ifremer.fr/thredds/dodsC/GLOBCURRENT-L4-CUREUL_HS-ALT_SUM-V03.0'
ocn_m = xa.open_dataset(URI)

OSError: [Errno -70] NetCDF: DAP server error: 'http://tds0.ifremer.fr/thredds/dodsC/GLOBCURRENT-L4-CUREUL_HS-ALT_SUM-V03.0'

## Data subsetting
Data can be subset in terms of 
1. variables (VOI: variables of interest)
2. horizontal extent (AOI: area of interest)
3. temporal extent (TOI: times of interest)

In [ ]:
# Bounding boxes for AOI 
bbox_idx = slice(700,900,1) #longitude AOI
bbox_idy = slice(180,270,1) #latitude AOI

fig,ax = plt.subplots(ncols=2,figsize=(15,7))
ax[0].pcolormesh(ocn_m.eastward_eulerian_current_velocity.isel(time=24))
ax[1].pcolormesh(ocn_m.eastward_eulerian_current_velocity.isel(time=800,lat=bbox_idy,lon=bbox_idx))

Now, we define the subsets for VOI and TOI

In [ ]:
# Variables of interest
VOI = [ 
    'eastward_eulerian_current_velocity',
    'northward_eulerian_current_velocity',
]

In [ ]:
TOI = slice(67230,67249)

## Performing data subset

In [ ]:
ocn_m_subset=ocn_m[VOI].isel(time=TOI,lon=bbox_idx,lat=bbox_idy)

In [ ]:
# Create data subset extent for plotting
ocn_m_subset_extent_lon = [ocn_m_subset.lon[0].values,ocn_m_subset.lon[-1].values,ocn_m_subset.lon[-1].values,ocn_m_subset.lon[0].values,ocn_m_subset.lon[0].values]
ocn_m_subset_extent_lat = [ocn_m_subset.lat[0].values,ocn_m_subset.lat[0].values,ocn_m_subset.lat[-1].values,ocn_m_subset.lat[-1].values,ocn_m_subset.lat[0].values]

## Approximate horizontal resolution
The horizontal resolution of the global GlobCurrent product is 0.25 degrees. Thus, the distance in x-direction vary with latitude. Since the horizontal resolution is very coarse, we make an approximation the horizontal resolution in x-direction:

In [ ]:
# Check the approximate dx for 0.25 deg resolution
# approximate radius of earth in km
R = 6373.0

lat1 = np.deg2rad(35)
lat2 = np.deg2rad(35)
lon1 = np.deg2rad(10.25)
lon2 = np.deg2rad(10)

dlon = lon2 - lon1
dlat = lat2 - lat1

a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

distance = R * c

print(f"Result: {distance} km")

# Ray tracing
Here, we create an approximate cartesian subset of the GlobCurrent product covering the area around the Agulhas current

In [ ]:
dx= 22000 # approximately
dy=28000 # approximately

x = np.arange(len(ocn_m_subset.lon))*dx
y = np.arange(len(ocn_m_subset.lat))*dy

In [ ]:
X0, XN = x[0], x[-1]
Y0, YN = y[0], y[-1]

# Resolution
nx = len(x)
ny = len(y)

# Number of rays
nb_wave_rays =60

# Duration (in seconds)
T = 62*3600
print("T={}h".format(T/3600))

# Number of discrete time steps
nt = 1200

In [ ]:
wave_period = 10 # [s]
theta0 = np.ones(nb_wave_rays)*1.35 # [rad]

# Initial position
ipx = np.linspace(2.0*1e6,2.25*1e6,nb_wave_rays) # initial position x
ipy = np.linspace(0.5,0.47,nb_wave_rays)*1e6 # initial position y

In [ ]:
# Due to the data format in GlobCurrent, we have to recast the time variables
tt = [np.datetime64(ocn_m_subset.time.values[i]) for i in range(ocn_m_subset.time.size)]

In [ ]:
# NOTE, we have to rename the lat and lon dimensions to x and y, respectively.

wt = Wave_tracing(U=ocn_m_subset.assign(time=np.array(tt)).eastward_eulerian_current_velocity.rename({'lon':'x','lat':'y'}),
                  V=ocn_m_subset.assign(time=np.array(tt)).northward_eulerian_current_velocity.rename({'lon':'x','lat':'y'}),
                       nx=nx, ny=ny, nt=nt,T=T,
                       dx=dx,dy=dy, nb_wave_rays=nb_wave_rays,
                       domain_X0=X0, domain_XN=XN,
                       domain_Y0=Y0, domain_YN=YN,temporal_evolution=True                       
                      )

# Set initial conditions
wt.set_initial_condition(wave_period=wave_period, 
                              theta0=theta0,ipx=ipx,ipy=ipy,
                              )
# Solve
wt.solve()

## Plotting

In [ ]:
fs=12
lw=1.5
alpha=0.6
fig, ax = plt.subplots(frameon=False,ncols=2,figsize=(15,6),#sharex=True,sharey=True,
                      )#subplot_kw={'projection': ccrs.Mercator(), 'facecolor':"gray"})
speed_global = np.sqrt(ocn_m.eastward_eulerian_current_velocity.isel(time=TOI.start)**2 + ocn_m.northward_eulerian_current_velocity.isel(time=TOI.start)**2)
speed = np.sqrt(wt.U**2 + wt.V**2)


cf0=ax[0].pcolormesh(ocn_m.lon, ocn_m.lat, speed_global ,
                           cmap=cmocean.cm.speed, vmax=2,
                 shading='auto')


ax[0].plot(ocn_m_subset_extent_lon,ocn_m_subset_extent_lat,lw=lw)


cf1=ax[1].pcolormesh(x, y, speed.isel(time=8),
                           cmap=cmocean.cm.speed, vmax=2,
                 shading='auto')

step=1
for i in range(0,wt.nb_wave_rays,step):
    if i == 0:
        ax[1].plot(wt.ray_x[i,:],wt.ray_y[i,:],'-',c='tab:red',alpha=alpha,lw=lw,label='Wave ray')
    else:
        ax[1].plot(wt.ray_x[i,:],wt.ray_y[i,:],'-',c='tab:red',alpha=alpha,lw=lw)

ax[1].set_xlim([1.8e6,3.0e6])
ax[1].set_ylim([0.4e6,1.4e6])

ax[1].legend(fontsize=fs,loc='lower right')
ax[1].axes.set_xticklabels([])
ax[1].axes.set_yticklabels([])

fig.tight_layout()

cb = fig.colorbar(cf1, ax=[ax[0],ax[1]], orientation='horizontal',shrink=0.6,pad=0.07)
cb.set_label('Current speed [m/s]',fontsize=fs)

fig.savefig('agulhas_POC.png',dpi=120)
